In [2]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# >> Visualization
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
from scipy.stats import chi2_contingency
from IPython.display import display
from itertools import combinations
from scipy.stats import kstest, spearmanr, pearsonr
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [4]:
data = pd.read_csv("spain.csv", index_col=0)
data.head()

,restaurant_link,restaurant_name,original_location,country,region,province,city,address,latitude,longitude,claimed,awards,popularity_detailed,popularity_generic,top_tags,price_level,price_range,meals,cuisines,special_diets,features,vegetarian_friendly,vegan_options,gluten_free,original_open_hours,open_days_per_week,open_hours_per_week,working_shifts_per_week,avg_rating,total_reviews_count,default_language,reviews_count_in_default_language,excellent,very_good,average,poor,terrible,food,service,value,atmosphere,keywords
320900,g10021880-d13763192,Taberna La Sacristia,"[""Europe"", ""Spain"", ""Andalucia"", ""Province of ...",Spain,Andalucia,Province of Seville,NaN,"Calle del Concejo 19, 41870 Aznalcollar Spain",37.51928,-6.26885,Unclaimed,NaN,#4 of 5 Restaurants in Aznalcollar,#4 of 6 places to eat in Aznalcollar,Spanish,NaN,NaN,NaN,Spanish,NaN,NaN,N,N,N,NaN,NaN,NaN,NaN,3.0,1.0,English,1.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
320901,g10021880-d15758746,Tasca el Capricho,"[""Europe"", ""Spain"", ""Andalucia"", ""Province of ...",Spain,Andalucia,Province of Seville,NaN,"Calle del 28 de Febrero 9, 41870 Aznalcollar S...",37.52065,-6.26822,Unclaimed,NaN,#3 of 5 Restaurants in Aznalcollar,#3 of 6 places to eat in Aznalcollar,Spanish,NaN,NaN,NaN,Spanish,NaN,NaN,N,N,N,NaN,NaN,NaN,NaN,5.0,2.0,All languages,2.0,2.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
320902,g10021880-d19332558,Bar Las Adelfas,"[""Europe"", ""Spain"", ""Andalucia"", ""Province of ...",Spain,Andalucia,Province of Seville,NaN,"Calle Perdon N° 23 Capilla de La Cruz, 41870 A...",37.52428,-6.27144,Claimed,NaN,#5 of 5 Restaurants in Aznalcollar,#5 of 6 places to eat in Aznalcollar,"Mediterranean, Spanish, Grill, Diner",NaN,NaN,"Breakfast, Lunch, Dinner, Brunch, Drinks","Mediterranean, Spanish, Grill, Diner, Dining bars",NaN,NaN,N,N,N,"{""Mon"": [], ""Tue"": [""19:30-23:45""], ""Wed"": [""1...",6.0,51.0,6.0,3.0,2.0,All languages,2.0,1.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN
320903,g10021880-d19468788,El Rincon nº 7,"[""Europe"", ""Spain"", ""Andalucia"", ""Province of ...",Spain,Andalucia,Province of Seville,NaN,"Calle del Concejo 60, 41870 Aznalcollar Spain",37.51714,-6.26860,Claimed,NaN,#1 of 5 Restaurants in Aznalcollar,#1 of 6 places to eat in Aznalcollar,"Mid-range, Steakhouse, Cafe, Spanish",€€-€€€,€2-€18,"Lunch, Dinner, Drinks","Steakhouse, Cafe, Dining bars, Spanish",NaN,NaN,N,N,N,"{""Mon"": [], ""Tue"": [], ""Wed"": [], ""Thu"": [""20:...",4.0,23.0,7.0,5.0,18.0,All languages,18.0,17.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
320904,g10021880-d19847377,Nuevo jacaranda,"[""Europe"", ""Spain"", ""Andalucia"", ""Province of ...",Spain,Andalucia,Province of Seville,NaN,"Calle del 28 de Febrero 2, 41870 Aznalcollar S...",37.52088,-6.26844,Claimed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Reservations,N,N,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df = data.copy() # hace una copia

In [6]:
df2= df["address"].str.split(pat="(\d{5})(.*)",expand=True)

In [7]:
df2.sample(5)

,0,1,2,3
392336,"Calle Bonavista, 2,",08012,Barcelona Spain,
435206,"Ctra de Olot N-260, Km 50.6,",17746,", Cabanelles Spain",
459278,"Rambla del Garraf, 90 Coolinary Market,",08812,", Sant Pere de Ribes Spain",
422467,"Avda. Puertas del Sur, Urb La Venencia local N...",11408,Jerez De La Frontera Spain,
358885,"Calle Strachan 12,",29015,Malaga Spain,


In [8]:
df3 = df2[1].str.split(pat="(\d{5})(.*)",expand=True)

In [9]:
df4 = df3[2].str.replace("Spain","").reset_index()

In [10]:
import re
# Función para dividir la dirección en dos partes por el código postal
def extraer_direccion(direccion):
    # Buscar "Spain" en la dirección y cortar hasta ese punto
    if "Spain" in direccion:
        direccion = direccion.split("Spain")[0].strip()
    
    # Usar regex para encontrar la parte antes de una coma o un dígito
    match = re.search(r'^[^,\d]+', direccion)
    if match:
        return match.group(0).strip()
    else:
        return direccion.strip()

df[['direccion_filtrada']] = df['address'].apply(extraer_direccion).apply(pd.Series)